<a href="https://colab.research.google.com/github/BalthazazarDuc/Data-Science-Machine-Learning/blob/Balt/Distilbert_sequence_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Import and dataset loading

In [1]:
!pip install transformers torch
!pip install accelerate
!pip install transformers[torch] accelerate --upgrade


In [2]:
import pandas as pd
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, Trainer, TrainingArguments
import torch
from torch.utils.data import Dataset
import accelerate
import transformers
print(accelerate.__version__)

0.29.3


In [3]:
train=pd.read_csv('https://raw.githubusercontent.com/BalthazazarDuc/Data-Science-Machine-Learning/main/training_data.csv?token=GHSAT0AAAAAACOZ4C73W6RMSSTLRC7G5PO6ZQ77H5Q')
test=pd.read_csv('https://raw.githubusercontent.com/BalthazazarDuc/Data-Science-Machine-Learning/main/unlabelled_test_data.csv?token=GHSAT0AAAAAACOZ4C73AOLJGUSBJSLVF63OZQ72D6A')


In [4]:
label_dict = {
    'A1': 0,
    'A2': 1,
    'B1': 2,
    'B2': 3,
    'C1': 4,
    'C2': 5
}

# Appliquer le mapping sur la colonne des étiquettes
train['difficulty'] = train['difficulty'].map(label_dict)

#Model setting

In [5]:
# Charger le tokenizer
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-multilingual-cased')


# Charger le modèle pré-entraîné
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-multilingual-cased', num_labels=6)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
# Fonction pour tokeniser les données
def encode(texts):
    return tokenizer(texts.tolist(), max_length=256, padding="max_length", truncation=True)

# Application de la fonction de tokenisation
train_encodings = encode(train['sentence'])
test_encodings = encode(test['sentence'])

In [7]:
class FrenchLevelDataset(Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        if self.labels is not None:
            item['labels'] = torch.tensor(self.labels[idx], dtype=torch.long)
        return item

    def __len__(self):
        return len(self.encodings['input_ids'])

# Création des instances de dataset
train_dataset = FrenchLevelDataset(train_encodings, train['difficulty'])
test_dataset = FrenchLevelDataset(test_encodings)


In [10]:
# Charger le modèle pré-entraîné
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-multilingual-cased', num_labels=6)  # Assurez-vous que le nombre de labels correspond au nombre de niveaux de difficulté

# Configuration des paramètres d'entraînement
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=1,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=32,
    warmup_steps=125,  # Adjusted based on fewer epochs
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=5,
    evaluation_strategy="epoch",
    gradient_accumulation_steps=2
)

# Création de l'objet Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset  # Vous pouvez utiliser une partie des données train pour la validation, si nécessaire
)

# Lancement de l'entraînement
trainer.train()

# Évaluation
trainer.evaluate()

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss
1,0.945000,No log


{'eval_runtime': 406.3609,
 'eval_samples_per_second': 2.953,
 'eval_steps_per_second': 0.094,
 'epoch': 1.0}

In [ ]:
#Arguments for longer training (almost 4h)

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    evaluation_strategy="epoch"
)

In [11]:
# Generate predictions for the test dataset
predictions = trainer.predict(test_dataset)
predicted_labels_indices = predictions.predictions.argmax(-1)  # Get the index of the max logit which is the predicted class


In [12]:
# Assuming the label dictionary used earlier was like this:
label_dict = {'A1': 0, 'A2': 1, 'B1': 2, 'B2': 3, 'C1': 4, 'C2': 5}
# Invert the dictionary to map back from indices to labels
index_to_label = {v: k for k, v in label_dict.items()}

# Map the predicted label indices to actual labels
predicted_labels = [index_to_label[idx] for idx in predicted_labels_indices]


In [13]:
import pandas as pd

# Assuming the test DataFrame is named 'test' and has a column 'sentence' for the input text
test['difficulty'] = predicted_labels
results_df = test[['id', 'difficulty']]


In [14]:
results_df.to_csv('predicted_results.csv', index=False)

In [15]:
results_df

,id,difficulty
0,0,C2
1,1,A2
2,2,B1
3,3,A2
4,4,C2
...,...,...
1195,1195,B1
1196,1196,A2
1197,1197,C2
1198,1198,B2
